In [1]:
pip install ultralytics opencv-python roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.6/913.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("visionproject-90zmt").project("the-last-final-merge-computer-vision")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to The-Last-Final-Merge-Computer-Vision-1 in yolov8:: 100%|██████████| 15243/15243 [00:01<00:00, 9045.60it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
import shutil

# Define hyperparameters in a class
class HyperParams:
    def __init__(self):
        self.model = "yolov8n.pt"  # Pre-trained YOLOv8 Nano model
        self.epochs = 100          # Total number of epochs
        self.batch_size = 32      # Number of images per batch
        self.image_size = 640     # Image size for training and validation
        self.learning_rate = 0.01 # Initial learning rate
        self.output_dir = "./runs/train"  # Directory to save results
        self.log_every = 5        # Save weights and logs every N epochs
        self.dataset_1_path = "/kaggle/working/The-Last-Final-Merge-Computer-Vision-1"
        self.combined_dataset_path = "./combined_dataset"
        self.yaml_path = "/kaggle/working/The-Last-Final-Merge-Computer-Vision-1/data.yaml"

# Initialize hyperparameters
params = HyperParams()

# Load YOLOv8 Model
model = YOLO(params.model)  # Load the pre-trained YOLOv8 model

# Train with logging and saving weights
results = model.train(
    data=params.yaml_path,       # Path to the dataset YAML file
    epochs=params.epochs,
    batch=params.batch_size,
    imgsz=params.image_size,
    lr0=params.learning_rate,
    save_period=params.log_every,  # Save weights every N epochs
    project=params.output_dir,    # Directory to save outputs
    name="yolo_training"          # Subdirectory for this run
)

# Plot training metrics
if hasattr(results, "metrics") and results.metrics:
    metrics = results.metrics

    # Check for keys and ensure data exists
    if "train/loss" in metrics and "val/loss" in metrics:
        if metrics["train/loss"] and metrics["val/loss"]:  # Ensure these aren't empty
            plt.figure(figsize=(12, 8))
            plt.plot(metrics["train/loss"], label="Train Loss")
            plt.plot(metrics["val/loss"], label="Validation Loss")
            plt.xlabel("Epochs")
            plt.ylabel("Loss")
            plt.title("Training and Validation Loss")
            plt.legend()
            plt.grid()
            plt.show()
        else:
            print("Train or validation loss data is missing or empty.")

    if "val/mAP_50" in metrics and "val/mAP_50-95" in metrics:
        if metrics["val/mAP_50"] and metrics["val/mAP_50-95"]:  # Ensure these aren't empty
            plt.figure(figsize=(12, 8))
            plt.plot(metrics["val/mAP_50"], label="mAP@50")
            plt.plot(metrics["val/mAP_50-95"], label="mAP@50-95")
            plt.xlabel("Epochs")
            plt.ylabel("mAP")
            plt.title("Mean Average Precision (mAP)")
            plt.legend()
            plt.grid()
            plt.show()
        else:
            print("mAP data is missing or empty.")
else:
    print("No metrics available in training results.")


# Output training results
print(f"Training complete. Results saved to: {results.save_dir}")


100%|██████████| 6.25M/6.25M [00:00<00:00, 84.4MB/s]


Ultralytics 8.3.68 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/The-Last-Final-Merge-Computer-Vision-1/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=./runs/train, name=yolo_training, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cr

100%|██████████| 755k/755k [00:00<00:00, 16.4MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 76.1MB/s]


...


100 epochs completed in 1.919 hours.
Optimizer stripped from runs/train/yolo_training/weights/last.pt, 6.3MB
Optimizer stripped from runs/train/yolo_training/weights/best.pt, 6.3MB

Validating runs/train/yolo_training/weights/best.pt...
Ultralytics 8.3.68 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:08<00:00,  2.60it/s]


                   all       1376       1151      0.969      0.949       0.97       0.84
                 Paper        361        374      0.978      0.952      0.977      0.841
                  Rock        336        389      0.967      0.946      0.962      0.814
              Scissors        344        388      0.961      0.949       0.97      0.866


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/train/yolo_training
No metrics available in training results.
Training complete. Results saved to: runs/train/yolo_training


In [4]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))


download_file('/kaggle/working/runs', 'out')

/kaggle/working/out.zip